In [3]:
from typing import List, Type, TypeVar, Optional
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
import tiktoken
import json
import time
import os
from dotenv import load_dotenv
load_dotenv()

# Replace 'YOUR_API_KEY' with your actual OpenAI API
T = TypeVar('T', bound=BaseModel)

def extract_multi_needle(schema: Type[T], haystack: str, example_needles: List[str]) -> List[T]:
    extracted_needles = []
    model_name = 'gpt-4o-mini'

    model = ChatOpenAI(
        model_name=model_name,
        temperature=0,
        api_key=os.environ.get("OPENAI_API_KEY")
    )

    # Initialize tokenizer
    encoding = tiktoken.encoding_for_model(model_name)

    # Token limits
    max_tokens_per_request = 16000
    max_tokens_for_response = 1000
    max_tokens_for_prompt = max_tokens_per_request - max_tokens_for_response

    # Tokenize the haystack
    haystack_tokens = encoding.encode(haystack)
    num_tokens = len(haystack_tokens)

    # Determine chunk size
    chunk_size = 4000

    # Split the haystack into chunks
    chunks = [
        encoding.decode(haystack_tokens[i:i + chunk_size])
        for i in range(0, num_tokens, chunk_size)
    ]

    # Prepare schema description
    schema_description = "Extract information according to the following schema:\n{\n"
    for field_name, field in schema.__fields__.items():
        field_descr = field.description or ''
        field_type = (
            field.annotation.__name__ if hasattr(field.annotation, '__name__') else str(field.annotation)
        )
        schema_description += f'  "{field_name}": "{field_descr} ({field_type})",\n'
    schema_description += "}\n"

    # Prepare examples
    examples_text = "Examples of the desired output format:\n"
    for example in example_needles:
        examples_text += f"- {example}\n"

    # System prompt
    system_prompt = "You are an AI language model that extracts structured data from text."

    # Process each chunk
    for idx, chunk in enumerate(chunks):
        user_prompt = (
            f"{schema_description}\n"
            f"{examples_text}\n"
            f"Text to analyze:\n\"\"\"\n{chunk}\n\"\"\"\n"
            "Extract any instances matching the schema from the text above. "
            "Provide the output as a JSON array of objects."
        )

        # Ensure the prompt fits within token limits
        prompt_tokens = encoding.encode(user_prompt)
        if len(prompt_tokens) > max_tokens_for_prompt:
            print(f"Prompt too long for chunk {idx}, reducing chunk size.")
            continue

        # Call OpenAI API
        try:
            response = model.invoke(
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
            )
            assistant_reply = response['choices'][0]['message']['content']

            # Parse the assistant's reply
            try:
                extracted_data = json.loads(assistant_reply)
                if isinstance(extracted_data, list):
                    for item in extracted_data:
                        try:
                            extracted_item = schema(**item)
                            extracted_needles.append(extracted_item)
                        except Exception as e:
                            print(f"Error parsing item: {item}, error: {e}")
                else:
                    print(f"Expected a list, got: {type(extracted_data)}")
            except json.JSONDecodeError as e:
                print(f"JSON decode error for chunk {idx}: {e}")
                print("Assistant reply:")
                print(assistant_reply)
        except Exception as e:
            print(f"Error with OpenAI API for chunk {idx}: {e}")
            if "rate limit" in str(e).lower():
                print("Rate limit exceeded. Sleeping for 60 seconds.")
                time.sleep(60)
                continue

    return extracted_needles


# Example usage
if __name__ == "__main__":
    class TechCompany(BaseModel):
        name: Optional[str] = Field(default=None, description="The full name of the technology company")
        location: Optional[str] = Field(default=None, description="City and country where the company is headquartered")
        employee_count: Optional[int] = Field(default=None, description="Total number of employees")
        founding_year: Optional[int] = Field(default=None, description="Year the company was established")
        is_public: Optional[bool] = Field(default=None, description="Whether the company is publicly traded (True) or privately held (False)")
        valuation: Optional[float] = Field(default=None, description="Company's valuation in billions of dollars")
        primary_focus: Optional[str] = Field(default=None, description="Main area of technology or industry the company focuses on")

    example_needles = [
        "Ryoshi, based in Neo Tokyo, Japan, is a private quantum computing firm founded in 2031, currently valued at $8.7 billion with 1,200 employees focused on quantum cryptography."
    ]

    # Sample haystack text
    haystack = """
    In the vast expanse of the universe, Ryoshi, based in Neo Tokyo, Japan, is a private quantum computing firm founded in 2031, currently valued at $8.7 billion with 1,200 employees focused on quantum cryptography.
    Another notable entity is SandTech, located in Arrakeen, Arrakis, a public nano-engineering corporation established in 2045, with a valuation of $12.5 billion and employing 3,500 people, specializing in nano-filtration systems.
    Amidst the dunes, SpiceCorp emerged in 2050, a private company headquartered in Sietch Tabr, employing 500 individuals dedicated to spice extraction technologies, currently valued at $2.1 billion.
    """

    extracted_data = extract_multi_needle(TechCompany, haystack, example_needles)

    for item in extracted_data:
        print(item)


Error with OpenAI API for chunk 0: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

